In [1]:
import requests
import os
import json
import csv
import pandas as pd
import datetime
import dateutil.parser
import unicodedata
import time

# Enter your path of data file here


In [2]:
file_path = 'Twitter-Dataset/data.csv'

# Enter bearer token

In [12]:
bearer_token = ''

# Change date here

In [4]:
# update the dates by 1 day each every day
dates = ['2022-02-13', '2022-02-14', '2022-02-15']

In [5]:
s_time = ['00:00:00', '01:00:00','02:00:00','03:00:00','04:00:00','05:00:00','06:00:00','07:00:00','08:00:00','09:00:00','10:00:00', '11:00:00', '12:00:00', '13:00:00', '14:00:00', '15:00:00', '16:00:00', '17:00:00', '18:00:00', '19:00:00', '20:00:00', '21:00:00', '22:00:00', '23:00:00'] 
e_time = ['01:00:00', '02:00:00','03:00:00','04:00:00','05:00:00','06:00:00','07:00:00','08:00:00','09:00:00','10:00:00','11:00:00', '12:00:00', '13:00:00', '14:00:00', '15:00:00', '16:00:00', '17:00:00', '18:00:00', '19:00:00', '20:00:00', '21:00:00', '22:00:00', '23:00:00','23:59:59']

In [6]:
def custom_create_url(tag, s_date, e_date):
  url = "https://api.twitter.com/2/tweets/search/recent"
  params = {
      'query': '#'+tag+' lang:en -is:retweet',
      'start_time': s_date,
      'end_time': e_date,
      'max_results':20,
      'sort_order': 'relevancy',
      'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld',
      'expansions': 'attachments.poll_ids,attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
      'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
      'media.fields': 'duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics,alt_text',
      'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
      'poll.fields': 'duration_minutes,end_datetime,id,options,voting_status'
  }
  return url, params

In [7]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r

In [8]:
def custom_connect_to_endpoint(custom_url, params):
    response = requests.request("GET", custom_url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [9]:
def make_csv(json_response, file_path, csvWriter):
  counter = 0
  
  if 'data' in json_response:
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # example = 
        # {
        #           "country": "United States",
        #           "country_code": "US",
        #           "full_name": "Chicago, IL",
        #           "geo": {
        #               "bbox": [
        #                   -87.940033,
        #                   41.644102,
        #                   -87.523993,
        #                   42.0230669
        #               ],
        #               "properties": {},
        #               "type": "Feature"
        #           },
        #           "id": "1d9a5370a355ab0c",
        #           "name": "Chicago",
        #           "place_type": "city"
        #       }

        geo = ""
        country = ""
        country_code = ""
        place_full_name = ""
        place_name = ""
        place_type = ""
        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
            for place in json_response['includes']['places']:
              if place['id'] == geo:
                country = place['country']
                country_code = place['country_code']
                place_full_name = place['full_name']
                place_name = place['name']
                place_type = place['place_type']
        else:
            geo = ""

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']

        # 9 hashtags
        tags = ''
        if ('hashtags' in tweet['entities']):  
          for tag in tweet['entities']['hashtags']:
            tags += tag['tag'] + str(',')
          tags = tags[:-1]

        #sensitive
        sensitive = tweet['possibly_sensitive']

        # urls for further analysis
        urls = ''
        if 'urls' in tweet['entities']:
          for url in tweet['entities']['urls']:
            urls += url['url'] + str(',')
          urls = urls[:-1]
        
        #annotations
        context_text = ''
        context_probability = 0
        context_type = ''
        if 'tweets' in json_response['includes']:
          for tweets_for_annotation in json_response['includes']['tweets']:
            if tweets_for_annotation['conversation_id'] == tweet['conversation_id']:
              if 'entities' in tweets_for_annotation:
                if 'annotations' in tweets_for_annotation['entities']:
                  for annotation in tweets_for_annotation['entities']['annotations']:
                    context_text = annotation['normalized_text']
                    context_probability = annotation['probability']
                    context_type = annotation['type']


        
        # Assemble all data in a list
        res = [author_id, created_at, geo, country, country_code, place_full_name, place_name, place_type, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text, tags, sensitive, urls, context_text, context_probability, context_type]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1




  # Print the number of tweets for this iteration
  print("# of Tweets added from this response: ", counter) 

# update query list every day

In [10]:
def main():
    # Updateee this listtttt
    query_list = ['canada', 'nasa']

    csvFile = open(file_path, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(['author id', 'created_at', 'geo', 'country', 'country_code', 'place_full_name', 'place_name', 'place_type', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet', 'hashtags', 'sensitive', 'urls', 'context_text', 'context_probability', 'context_type'])
    for date in dates:
      for i in range(len(s_time)):
        # url = create_url()
        # json_response = connect_to_endpoint(url)
        # 'start_time': '2022-02-16T00:00:00.000Z',
        cur_s_time = date+'T'+s_time[i]+'.000Z'
        cur_e_time = date+'T'+e_time[i]+'.000Z'
        print(cur_s_time, cur_e_time)
        for tag in query_list: 
          custom_url, params = custom_create_url(tag, cur_s_time, cur_e_time)
          custom_json_response = custom_connect_to_endpoint(custom_url, params)
          make_csv(custom_json_response, file_path, csvWriter)
          # print(json.dumps(custom_json_response, indent=4, sort_keys=True))
    
    # When done, close the CSV file
    csvFile.close()
    

In [11]:
if __name__ == "__main__":
    main()

2022-02-13T00:00:00.000Z 2022-02-13T01:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  18
2022-02-13T01:00:00.000Z 2022-02-13T02:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  18
2022-02-13T02:00:00.000Z 2022-02-13T03:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-13T03:00:00.000Z 2022-02-13T04:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  15
2022-02-13T04:00:00.000Z 2022-02-13T05:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  14
2022-02-13T05:00:00.000Z 2022-02-13T06:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-13T06:00:00.000Z 2022-02-13T07:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-0

200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-15T11:00:00.000Z 2022-02-15T12:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  19
2022-02-15T12:00:00.000Z 2022-02-15T13:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-15T13:00:00.000Z 2022-02-15T14:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-15T14:00:00.000Z 2022-02-15T15:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-15T15:00:00.000Z 2022-02-15T16:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-15T16:00:00.000Z 2022-02-15T17:00:00.000Z
200
# of Tweets added from this response:  20
200
# of Tweets added from this response:  20
2022-02-15T17:00:00.000Z 2022-02-15T18:00:00.000Z
200
# 